In [1]:
import pandas as pd

# query = "Query_NYT_5y_surveillance"
# query = "Query_NYT_13y_surveillance"
query = "Query_US_5y_Svalbard"

CSV_FILE = query + ".csv"
FEATHER_FILE = query + ".feather"

df = pd.read_feather(FEATHER_FILE)
df = df.drop(df[df.text_downloaded == False].index)
df = df.reset_index(drop=True)

df.sample(3)

,stories_id,title,url,year,text,text_downloaded,tags,GPE,IND,ORG
452,1931029939,500-Year-Old 'Mad' Sailor Shop Arthur Beale Su...,https://www.wsj.com/articles/500-year-old-mad-...,2021,Dan Barker had barely finished rejoicing that ...,True,{'allDescriptors': [{'label': 'bombs and explo...,"[London, London, London, London, London, Londo...","[Dan Barker, Arthur Beale, Arthur Beale, Arthu...","[Arthur Beale Ltd., James Smith & Sons, Marwoo..."
182,1531953980,Human disturbance increasing cannibalism among...,https://www.theguardian.com/world/2020/feb/26/...,2020,Cases of polar bears killing and eating each o...,True,"{'allDescriptors': [{'label': 'animals', 'scor...","[Moscow, St Petersburg, cubs, Russia, St Peter...","[Ilya Mordvintsev, Yamal, Vladimir Sokolov, So...","[Interfax, Mordvintsev, Severtsov Institute of..."
409,1862134071,The Guardian view on Radio Garden: world citiz...,https://www.theguardian.com/commentisfree/2021...,2021,"In The Aleph, a short story by Jorge Luis Borg...",True,{'allDescriptors': [{'label': 'computers and t...,"[Aleph, Buenos Aires, Amsterdam, Tralee, west ...","[Jorge Luis Borges, Bertoua, Borges]","[Radio Garden, Radio Kerry, Radio Blanc-Sablon..."


In [9]:
import requests
import json
from tqdm.notebook import trange  # to display a progress bar
import ipywidgets as widgets

# Tests
# curl -X POST http://localhost:8000/predict.json -H "Content-Type: application/json" -d '{"text": "Federal agents show stronger force at Portland protests despite order to withdraw" }'
# curl -X POST https://nyt-news-labeler-qoqojvpyua-ez.a.run.app/predict.json -H "Content-Type: application/json" -d '{"text": "Federal agents show stronger force at Portland protests despite order to withdraw" }'

# URL = 'http://nyt-news-labeler:8000'
URL = "https://nyt-news-labeler-qoqojvpyua-ez.a.run.app/"

headers = {"Content-Type": "application/json"}

tags = [{}] * len(df)

for index in trange(0, len(df)):

    text = df.loc[index].text
    text = text.replace("\n", "")

    try:
        data = {"text": text}  # article's text

        response = requests.post(
            URL + "predict.json", data=json.dumps(data), headers=headers
        )

        _json = json.loads(response.text)

        tags[index] = _json["predictions"]['descriptors3000']

    except Exception as e:
        print(tags[index])

df["tags"] = tags


  0%|          | 0/1060 [00:00<?, ?it/s]

## Save Dataset

In [10]:
# df = df.drop(df[df.tags == {}].index)
# df = df.reset_index(drop=True)

df.to_feather(FEATHER_FILE)